# Tokenisation avec des utilitaires en ligne de commande

Sans être exclusive, la liste à suivre présente quelques outils pour segmenter du texte. Le choix s’effectuera souvent selon la langue de travail.

## GREP

L’utilitaire `grep` (*file pattern searcher*) associé à l’option `-o` n’affiche que les segments où le motif en paramètre a été repéré :

In [ ]:
# whitespace tokenization
!echo 'Le petit chat boit du lait.' | grep -o '\S\+'

# tokenization based on punctuation
!echo 'Le petit chat, rentré de sa promenade, boit du lait.' | grep -o '[^[:punct:]]\+'

La commande fonctionne également depuis un fichier qui lui serait transmis :

In [ ]:
!grep -o '\S\+' ./files/cnrs.txt

Comme `grep` est conçue pour de la recherche de motifs, des solutions plus adaptées existent pour effectuer une segmentation sur des règles plus complexes que l’espace ou la ponctuation.

## SED

`sed` (*stream editor*) est un utilitaire très puissant qui permet d’éditer les lignes d’un flux en effectuant des remplacements (`s`), des suppressions (`d`) ou des insertions (`a`, `i`). Il repose sur l’utilisation des expressions rationnelles.

### Fonctionnement général de la commande de substitution

Pour remplacer la première occurrence d’un motif :

In [ ]:
# substitutes first occurrence of 'id' by 'it'
!echo 'Le petit chat boid du laid.' | sed 's/id/it/'

Pour cette fois-ci effectuer les remplacements autant de fois que possible :

In [ ]:
!echo 'Le petit chat boid du laid.' | sed 's/id/it/g'

### Segmentation sur l’espace

La segmentation sur l’espace (*whitespace tokenization*) peut se commander en demandant de lui substituer des retours à la ligne :

In [ ]:
!echo "Le petit chat boit du lait." | sed 's/ /\n/g'

Toutefois, la version GNU `sed` sous Linux ne traite pas le caractère `\n` comme un saut de ligne dans les remplacements. Une syntaxe alternative qui fonctionne également avec BSD `sed` sous MacOS recourt au caractère spécial `&` :

In [ ]:
# whitespace tokenization on GNU sed
!echo "Le petit chat boit du lait." | sed 's/ /\'$'\n''/g'

Pour indiquer d’effectuer la segmentation sur un fichier en paramètre :

In [ ]:
!sed 's/ /\'$'\n''/g' ./files/cnrs.txt

Si l’utilitaire renvoie ordinairement le flux dans la sortie standard, l’option `-i`, accompagnée d’une extension pour sauvegarder l’original, effectue le remplacement en place dans les fichiers en paramètres :

In [ ]:
!sed -i.bak 's/ /\'$'\n''/g' ./files/cnrs.txt

### Autres règles de segmentation

`sed` acceptant les expressions rationnelles, il est possible de concevoir des règles de segmentation personnalisées en faisant attention aux différentes versions implémentées dans les architectures (BSD ou GNU) :

In [ ]:
# comma-based tokenization
!sed 's/,/\n/g' ./files/cnrs.txt

# tokenization based on non-alphanumeric characters
# (including punctuation)
!sed 's/[^[:alnum:]]\+/\n/g' ./files/cnrs.txt
    
# tokenization by capital letters
# (useful for camelCase or sentence-like segmentation)
!sed 's/\([A-Z]\)/\n\1/g' ./files/cnrs.txt

# tokenization by word length
!sed 's/\b\w\{5,\}\b/&\n/g' ./files/cnrs.txt

# tokenization by spaces and punctuationetc.)
!sed 's/[[:space:][:punct:]]\+/\n/g' ./files/cnrs.txt

## AWK

`awk` est un langage qui permet de traiter des fichiers plats par lignes. Il est le plus souvent employé comme analyseur de fichiers structurés afin de renvoyer les informations de champs spécifiques :

In [ ]:
# print the first field,
# considering the semi-colon as the field separator
!echo 'adj;Petite;petit' | awk -F ';' '{print $1}'

# print 'Petite : adj'
!echo 'adj;Petite;petit' | awk -F ';' '{print $2, ":", $1}'

# another example of a formatted output
!echo 'adj;Petite;petit' | \
  awk -F ';' '{print "cat:", $1, "\nword:", $2, "\nlemma:", $3}'

En détournant cet usage, il est possible de confectionner un petit outil de segmentation basé sur l’espace :

In [ ]:
!echo 'Le petit chat boit du lait.' | \
  awk '{for (i = 1; i <= NF; i++) print $i}'

Avec prise en charge d’un fichier en entrée, l’exemple donne :

In [ ]:
!awk '{for (i = 1; i <= NF; i++) print $i}' ./files/cnrs.txt

**Remarques :** Sans l’option `-F`, `awk` considère l’espace ou la tabulation comme caractère séparateur des champs. Et quant à la variable prédéfinie `NF`, elle représente le nombre de champs dans la ligne courante.

Pour segmenter également sur les signes de ponctuation, on peut utiliser la sous-commande `gsub` pour insérer des espaces autour de ces caractères à l’aide d’une expression rationnelle qui permet de traiter les signes de ponctuation comme des éléments distincts lors de la segmentation :

In [ ]:
!awk '{\
    gsub(/([[:punct:]])/, " \1 "); \
    for (i = 1; i <= NF; i++) \
        print $i \
}' ./files/cnrs.txt

Sur MacOS, l’interprétation des constructions de `awk` est parfois différente. Une expression plus portable devient :

In [ ]:
!awk '{\
  gsub(/([[:punct:]])/, " & ");\
  for (i = 1; i <= NF; i++)\
    print $i\
}' ./files/cnrs.txt

La prise en charge des expressions rationnelles permet de créer des règles personnalisées, qu’il conviendra toujours d’adapter en fonction des plateformes si elles ne fonctionnent pas telles quelles :

In [ ]:
# comma-based tokenization
!awk '{gsub(",", "\n"); print}' ./files/cnrs.txt

# tokenization based on non-alphanumeric characters
# (punctuation included)
!awk '{gsub(/[^[:alnum:]]+/, "\n"); print}' ./files/cnrs.txt

# tokenization by capital letters
!awk '{gsub(/[A-Z]/, " &"); gsub(/ +/, "\n"); print}' ./files/cnrs.txt

# tokenization by spaces and punctuation
!awk '{gsub(/[^[:alnum:]]+/, "\n"); print}' ./files/cnrs.txt

## TR

Un autre utilitaire pour manipuler le flux d’un texte est `tr` (*translate characters*). Il offre la capacité d’effectuer une concordance entre plusieurs caractères à remplacer :

In [ ]:
# 'a' => 'e' ; 'e' => 'a' ; 't' => 'z'
!echo 'Le petit chat boit du lait.' | tr 'aet' 'eaz'

La segmentation sur l’espace s’effectue plus simplement qu’avec `sed` :

In [ ]:
!echo 'Le petit chat boit du lait.' | tr ' ' '\n'

Les remplacements peuvent naturellement s’opérer depuis un fichier :

In [ ]:
!tr ' ' '\n' < ./files/cnrs.txt

En combinant les options `-c` (remplacement sur tous les autres caractères sauf ceux indiqués) et `-s` (fusion des caractères répétés), il est possible d’appliquer d’autres règles de segmentation :

In [ ]:
# comma-based tokenization
!tr ',' '\n' < ./files/cnrs.txt

# tokenization based on non-alphanumeric characters
# (punctuation included)
!tr -cs '[:alnum:]' '\n' < ./files/cnrs.txt

# tokenization by capital letters
!tr 'A-Z' ' ' < ./files/cnrs.txt | tr -s ' ' '\n'

# tokenization by spaces and punctuation
!tr -cs '[:alnum:]' '\n' < ./files/cnrs.txt